In [1]:
import cv2
import dlib
from scipy.spatial import distance as dist
from threading import Thread
import imutils
from imutils import face_utils
from imutils.video import FileVideoStream
from imutils.video import  WebcamVideoStream

In [2]:
def ear(eye):    # eye aspect ratio
    vertical1_dist = dist.euclidean(eye[1], eye[5])
    vertical2_dist = dist.euclidean(eye[2], eye[4])
    horizontal_dist = dist.euclidean(eye[0], eye[3])
    aspect_ratio = (vertical1_dist + vertical2_dist) / (2.0 * horizontal_dist)
    return aspect_ratio

In [3]:
threshold = 0.325
counter =  0
framenos = 3
#blinks = 0

In [4]:
predictor_path = 'shape_predictor_68_face_landmarks.dat'
predictor = dlib.shape_predictor(predictor_path)
detector = dlib.get_frontal_face_detector()

In [5]:
(lx,ly) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rx,ry) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

In [6]:
#file = FileVideoStream('happy.mp4').start()
file =  WebcamVideoStream(src=0).start()

In [7]:
first_frame = file.read()
(iheight,iwidth) = first_frame.shape[:2]
print(iheight,iwidth)

480 640


In [8]:
#fourcc = cv2.VideoWriter_fourcc(*'XVID')
#output = cv2.VideoWriter('test_output.mp4',fourcc, 10,(iwidth,iheight))

In [9]:
def det_to_bb(det):
    x = det.left()
    y = det.top()
    w = det.right() 
    h = det.bottom() 
    return (x, y, w, h)

In [10]:
while True:
    frame = file.read()
    frame = imutils.resize(frame, width=600)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    detect = detector(gray)
    if (len(detect)>0) :
        for (i,det) in enumerate(detect):
            ishape = predictor(gray,det)
            ishape = face_utils.shape_to_np(ishape)
            (x, y, w, h) = face_utils.rect_to_bb(det)
            cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),3)
            leftEye = ishape[lx:ly]
            rightEye = ishape[rx:ry]

            leftEar = ear(leftEye)
            rightEar = ear(rightEye)
            avgEar = (leftEar+rightEar)/2
            if avgEar < threshold:
                counter += 1
                if counter >= framenos:
                    #blinks += 1
            #cv2.putText(frame, " blinks = {}".format(blinks),(x,y),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
            #if (blinks >=1):
                    cv2.putText(frame, "DROWSY",(x,y),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 2)
            else:
                counter = 0
                cv2.putText(frame, "NOT DRWOSY",(x,y),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
    else:
        cv2.putText(frame, "NO FACE PRESENT",(20,20),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
    cv2.imshow('frame',frame)
    frame = cv2.resize(frame,(iwidth,iheight))
    #output.write(frame)
    key = cv2.waitKey(1)
    if key == ord('q'):
        file.stop()
        break

#output.release()
file.stream.release()
cv2.destroyAllWindows()